# OpenAPI to OpenAI Assistant Function Mapping: CoinGecko API Example

This notebook demonstrates how to leverage an [OpenAPI specification](https://github.com/OAI/OpenAPI-Specification) with the OpenAI Assistant API to map CoinGecko's API (or any other OpenAPI) and utilize [Function Calling](https://platform.openai.com/docs/guides/function-calling). We'll stream the response from the OpenAI API while creating reusable classes that can easily be adapted for different Assistants or OpenAPI specs.

To explore more OpenAPI specifications for your projects, consider visiting [API.guru](https://apis.guru/).



### Step 1: Mapping and Calling OpenAPI Endpoints Dynamically

This code provides a framework for mapping OpenAPI specifications to Python classes and calling API endpoints dynamically. Here's a summary:
- Operation Class: Represents an individual API operation, including the HTTP method, URL template, and any required path or query parameters. It provides methods for constructing the URL dynamically based on the parameters provided.
- OpenAPISpecMapper Class: Parses an OpenAPI specification to create a mapping of operations (by their unique operationId). Each operation is represented as an Operation object, providing a structured way to access and call these endpoints. The call_api method allows calling a specific API operation using its operationId, constructing the appropriate URL and passing any necessary parameters or data.

This structure abstracts API calls and provides a clean, reusable approach to interact with an API described via an OpenAPI specification.

In [2]:
from typing import Dict, List

import jsonref
import requests


class Operation:
    def __init__(self, method: str, url_template: str, operation_id: str, path_params=None, query_params=None):
        self.method = method
        self.url_template = url_template
        self.operation_id = operation_id
        self.path_params = path_params if path_params else []
        self.query_params = query_params if query_params else []

    def __repr__(self):
        return f"Operation(method={self.method}, url_template={self.url_template}, operation_id={self.operation_id})"

    def construct_url(self, **kwargs):
        url = self.url_template.format(**{k: v for k, v in kwargs.items() if k in self.path_params})
        query_string = "&".join([f"{k}={v}" for k, v in kwargs.items() if k in self.query_params])
        if query_string:
            url += "?" + query_string
        return url


class OpenAPISpecMapper:
    def __init__(self, openapi_spec):
        self.operation_mapping = self.openapi_to_operation_mapping(openapi_spec)

    def openapi_to_operation_mapping(self, openapi_spec) -> Dict[str, Operation]:
        operation_mapping: Dict[str, Operation] = {}
        server = openapi_spec["servers"][0]["url"]

        for path, methods in openapi_spec["paths"].items():
            for method, spec_with_ref in methods.items():
                spec = jsonref.replace_refs(spec_with_ref)
                operation_id = spec.get("operationId")
                if not operation_id:
                    continue
                path_params, query_params = self._extract_params(spec.get("parameters", []))
                operation_mapping[operation_id] = Operation(method=method.upper(),
                                                            url_template=f"{server}{path}",
                                                            operation_id=operation_id,
                                                            path_params=path_params,
                                                            query_params=query_params)
        return operation_mapping

    def _extract_params(self, params) -> (List[str], List[str]):
        path_params = [param["name"] for param in params if param.get("in") == "path"]
        query_params = [param["name"] for param in params if param.get("in") == "query"]
        return path_params, query_params

    def call_api(self, operation_id, parameters=None, data=None):
        operation = self.operation_mapping.get(operation_id)
        if not operation:
            raise ValueError("Operation ID not found")

        url = operation.construct_url(**(parameters or {}))
        response = requests.request(operation.method, url, json=data)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": "API call failed", "status_code": response.status_code}

In [3]:
def example_usage():
    with open('./coingecko_api_v3_openapi.json', 'r') as f:
        openapi_spec = jsonref.loads(f.read())

    mapper = OpenAPISpecMapper(openapi_spec)
    response = mapper.call_api("checkApiServerStatus")
    print(response)
example_usage()

{'gecko_says': '(V3) To the Moon!'}


### Step 2: Converting OpenAPI Specs to Functions for OpenAI Assistant

This code defines a class to convert OpenAPI specifications into function definitions compatible with OpenAI's API. Here's a summary:

OpenAPIToFunctionMapper Class:
- Purpose: Converts OpenAPI file data to a format suitable for OpenAI's Assistant API function calls.
- __init__ Method:
    - Takes an OpenAPI specification (in dictionary form) as input and stores it for processing.
- openapi_to_functions Method: Extracts each API operation from the OpenAPI specification, resolving references and constructing function definitions for OpenAI.
- Output: Returns a list of function definitions. Each function is represented as a dictionary containing its name, description, and expected parameters (both request bodies and URL/query parameters).
The resulting list can be used to seamlessly integrate OpenAPI-defined API endpoints into OpenAI's Function Calling feature, allowing OpenAI's assistant to call these functions directly.

In [4]:
from typing import List

import jsonref
from openai.types import FunctionDefinition


class OpenAPIToFunctionMapper:
    """
    Converts OpenAPI file to functions suitable for the OpenAI Assistant API
    """

    def __init__(self, openapi_spec):
        """
        Initialize the mapper with an OpenAPI specification.

        :param openapi_spec: The OpenAPI specification as a dictionary.
        """
        self.openapi_spec = openapi_spec

    def openapi_to_functions(self) -> List[FunctionDefinition]:
        """
        Converts the OpenAPI specification to a list of function representations for the OpenAI assistant.

        :return: A list of dictionaries, each representing a function.
        """
        functions = []

        for path, methods in self.openapi_spec["paths"].items():
            for method, spec_with_ref in methods.items():
                # Resolve JSON references.
                spec = jsonref.replace_refs(spec_with_ref)

                # Extract a name for the functions.
                function_name = spec.get("operationId")

                # Extract a description and parameters.
                desc = spec.get("description") or spec.get("summary", "")

                schema = {"type": "object", "properties": {}}

                req_body = (
                    spec.get("requestBody", {})
                    .get("content", {})
                    .get("application/json", {})
                    .get("schema")
                )
                if req_body:
                    schema["properties"]["requestBody"] = req_body

                params = spec.get("parameters", [])
                if params:
                    param_properties = {
                        param["name"]: param["schema"]
                        for param in params
                        if "schema" in param
                    }
                    schema["properties"]["parameters"] = {
                        "type": "object",
                        "properties": param_properties,
                    }

                if function_name:
                    functions.append(
                        {"name": function_name, "description": desc, "parameters": schema}
                    )

        return functions


In [5]:
def example_usage():
    # Load the OpenAPI spec from a file
    with open('./coingecko_api_v3_openapi.json', 'r') as f:
        openapi_spec = jsonref.loads(f.read())

    mapper = OpenAPIToFunctionMapper(openapi_spec)
    functions = mapper.openapi_to_functions()

    # Print the functions for demonstration
    import json

    print(json.dumps(functions, indent=4))
example_usage()

[
    {
        "name": "checkApiServerStatus",
        "description": "Check API server status\n",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    },
    {
        "name": "getCurrentCryptocurrencyPrice",
        "description": "Note: to check if a price is stale, please flag `include_last_updated_at=true` to get the latest updated time. You may also flag `include_24hr_change=true` to check if it returns 'null' value.\n\nCache / Update Frequency: every 60 seconds  (every 30 seconds for Pro API)",
        "parameters": {
            "type": "object",
            "properties": {
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ids": {
                            "type": "string"
                        },
                        "vs_currencies": {
                            "type": "string"
                        },
                        "include_market_

### Step 3: Adding OpenAPI Functions to Your OpenAI Assistant

This snippet demonstrates how to integrate OpenAPI specifications directly into an OpenAI assistant. Here's a brief description:

AssistantManager Class:
- Purpose: Manages an OpenAI assistant by allowing OpenAPI specifications to be incorporated as tools.
- __init__ Method: Initializes the manager with the OpenAI client using the provided API key.
- add_openapi_spec_to_assistant Method:
    - Takes an OpenAI assistant ID and a file path to an OpenAPI specification as inputs.
    - Loads and parses the OpenAPI specification.
    - Utilizes OpenAPIToFunctionMapper to convert API definitions to function tools for OpenAI.
    - Updates the specified OpenAI assistant's tools to include these new functions.

This method simplifies the addition of OpenAPI-defined functions to an OpenAI assistant, making API interactions seamless.

In [ ]:
import jsonref
from openai import OpenAI
from openai.types.beta import FunctionToolParam

class AssistantManager:
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)

    def add_openapi_spec_to_assistant(self, assistant_id: str, path_to_openapi_spec: str):
        with open(path_to_openapi_spec, 'r') as f:
            openapi_spec = jsonref.loads(f.read())
        # Create an instance of the mapper with the loaded OpenAPI spec
        mapper = OpenAPIToFunctionMapper(openapi_spec)
        # Convert the OpenAPI spec to functions
        functions = mapper.openapi_to_functions()

        function_tool_params: List[FunctionToolParam] = []
        for f in functions:
            function_tool_params.append({
                "function": f,
                "type": "function"
            })

        self.client.beta.assistants.update(assistant_id=assistant_id, tools=function_tool_params)

In [ ]:
def example_usage():
    assistant_manager = AssistantManager("<your-openai-api-key>")
    assistant_manager.add_openapi_spec_to_assistant("asst_<your-assistant-id>", "./coingecko_api_v3_openapi")
    

### Step 4: Managing and Streaming Conversations with OpenAI Assistants

This code snippet defines a manager for handling message threads and streaming responses with an OpenAI assistant. Here's a brief description:

StreamingThreadManager Class:
Purpose: Manages OpenAI assistant threads, allowing for message exchange and real-time streaming.
- __init__ Method: Initializes the manager with an API key, an assistant ID, and an optional OpenAPI spec mapper for API calls.
- Thread Creation and Retrieval:
    - create_thread: Creates a new assistant thread.
    - retrieve_thread: Retrieves an existing thread by its ID.
- submit_message Method: Sends a user message to a thread, optionally attaching file IDs.
- stream_single_run Method:
    - Streams responses from a thread in real time using an event handler.
    - If the assistant requires further action (e.g., calling API tools), it uses the OpenAPI spec mapper to fulfill those requests and submits the tool outputs back to the assistant, streaming the final output to the user.


This framework helps facilitate seamless conversations with OpenAI assistants while enabling them to leverage API tools in real time.

In [10]:
from typing import Optional, Iterator
from openai import OpenAI
from openai.lib.streaming import AssistantEventHandler
from openai.types.beta import Thread

import json


class StreamingThreadManager:
    def __init__(self, api_key: str, assistant_id: str, openapi_spec_mapper: Optional[OpenAPISpecMapper] = None):
        self.client = OpenAI(api_key=api_key)
        self.assistant_id = assistant_id
        self.open_api_spec_mapper = openapi_spec_mapper

    def create_thread(self) -> Thread:
        return self.client.beta.threads.create()

    def retrieve_thread(self, thread_id: str) -> Thread:
        return self.client.beta.threads.retrieve(thread_id)

    def submit_message(self, thread, user_message, attachments=None):
        if attachments is None:
            attachments = []

        self.client.beta.threads.messages.create(
            thread_id=thread.id, role="user", content=user_message, attachments=attachments
        )

    def stream_single_run(self, thread: Thread) -> Iterator[str]:
        event_handler = AssistantEventHandler()
        with self.client.beta.threads.runs.stream(
                thread_id=thread.id,
                assistant_id=self.assistant_id,
                event_handler=event_handler
        ) as stream:
            for text in stream.text_deltas:
                yield text
            stream.until_done()

        if event_handler.current_run.status == 'requires_action' and self.open_api_spec_mapper is not None:
            tool_calls = event_handler.current_run.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for tool_call in tool_calls:
                operation_id = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                # Ensure parameters key exists
                arguments["parameters"] = arguments.get("parameters", arguments)
                response = self.open_api_spec_mapper.call_api(operation_id, parameters=arguments["parameters"])
                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "output": json.dumps(response)
                })

            with self.client.beta.threads.runs.submit_tool_outputs_stream(
                    thread_id=thread.id,
                    run_id=event_handler.current_run.id,
                    tool_outputs=tool_outputs,
            ) as stream:
                for text in stream.text_deltas:
                    yield text
                stream.until_done()

### Step 5: Practical Example: Streaming Crypto Data Queries Using OpenAI Assistant
This code demonstrates an example of how to use the previously defined classes and tools to interact with an OpenAI assistant using an OpenAPI specification.

In [11]:
import jsonref

OPENAI_API_KEY = '<your-openai-api-key>'
COIN_GECKO_ASSISTANT_ID = "<your-assistant-id>"
COIN_GECKO_OPENAPI_SPEC = './coingecko_api_v3_openapi.json'

# Example Usage:
with open(COIN_GECKO_OPENAPI_SPEC, 'r') as f:
    openapi_spec = jsonref.loads(f.read())

mapper = OpenAPISpecMapper(openapi_spec)
thread_manager = StreamingThreadManager(api_key=OPENAI_API_KEY, assistant_id=COIN_GECKO_ASSISTANT_ID,
                                        openapi_spec_mapper=mapper)

In [16]:
thread = thread_manager.create_thread()
thread_manager.submit_message(thread, "Which companies hold bitcoin?")

for x in thread_manager.stream_single_run(thread):
    print(x)

Here
 are
 some
 of
 the
 companies
 that
 hold
 Bitcoin
,
 along
 with
 their
 respective
 holdings
 and
 value
:


1
.
 **
Micro
Strategy
 Inc
.
**

  
 -
 Symbol
:
 NAS
DAQ
:M
STR


  
 -
 Country
:
 US


  
 -
 Total
 holdings
:
 
174
,
530
 BTC


  
 -
 Total
 current
 value
:
 $
11
,
188
,
882
,
638
 USD



2
.
 **
Gal
axy
 Digital
 Holdings
**

  
 -
 Symbol
:
 T
SE
:
 GL
XY


  
 -
 Country
:
 US


  
 -
 Total
 holdings
:
 
17
,
518
 BTC


  
 -
 Total
 current
 value
:
 $
1
,
123
,
055
,
326
 USD



3
.
 **
Mar
athon
 Digital
 Holdings
**

  
 -
 Symbol
:
 NAS
DAQ
:M
ARA


  
 -
 Country
:
 US


  
 -
 Total
 holdings
:
 
13
,
716
 BTC


  
 -
 Total
 current
 value
:
 $
879
,
314
,
239
 USD



4
.
 **
Tesla
,
 Inc
.
**

  
 -
 Symbol
:
 NAS
DAQ
:T
SL
A


  
 -
 Country
:
 US


  
 -
 Total
 holdings
:
 
10
,
500
 BTC


  
 -
 Total
 current
 value
:
 $
673
,
140
,
822
 USD



5
.
 **
H
ut
 
8
 Mining
 Corp
**

  
 -
 Symbol
:
 NAS
DAQ
:H
UT


  
 -
 Country
:
 CA


  
 -
 To